In [17]:
from database.market import Market
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.adatabase import ADatabase
import pickle

In [2]:
strat = ADatabase("cfa")

In [3]:
market = Market()
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()

In [4]:
factors=['assets', 'liabilitiesandstockholdersequity',
       'earningspersharebasic', 'earningspersharediluted',
       'stockholdersequity', 'netincomeloss', 'assetscurrent',
       'liabilitiescurrent','adjclose']

In [5]:
year = 2022
quarter = 2
projection_value = 4
training_year = 1

In [8]:
ds = []
market.connect()
for ticker in sp500["Symbol"]:
    try:
        filing = market.retrieve_high_level_filings(ticker)
        filing = p.column_date_processing(filing)
        filing.sort_values("date",inplace=True)
        filing["y"] = filing['adjclose'].shift(-projection_value)
        ds.append(filing)
    except Exception as e:
        continue
market.disconnect()
rekt = pd.concat(ds)
rekt.dropna(inplace=True)
strat.connect()
start_year = int(year - projection_value/4)
end_year = start_year + 1
for year in tqdm(range(start_year,int(end_year))):
    try:
        training_set = rekt[(rekt["year"]<year) & (rekt["year"]>=year-training_year)].reset_index()
        prediction_set = rekt[(rekt["year"]==year)]
        refined = {"X":training_set[factors],"y":training_set["y"]}
        models = m.regression(refined)
        for row in models.iterrows():
            try:
                model = row[1]["model"]
                api = row[1]["api"]
                prediction_set[f"{api}_prediction"] = model.predict(prediction_set[factors])
            except Exception as e:
                print(str(e))
        prediction_set["prediction"] = (prediction_set["skl_prediction"] + prediction_set["xgb_prediction"] + prediction_set["cat_prediction"]) / 3
        prediction_set["new_quarter"] = [(row[1]["quarter"] + projection_value) % 4 for row in prediction_set.iterrows()]
        prediction_set["new_quarter"] = prediction_set["new_quarter"].replace(0,4)
        prediction_set["new_year"] = [row[1]["year"] if row[1]["quarter"] + projection_value <=4 \
                                      else row[1]["year"]+ projection_value/4 for row in prediction_set.iterrows()]
        prediction_set = prediction_set[["new_year","new_quarter","ticker","prediction"]].rename(columns={"new_year":"year","new_quarter":"quarter"})
        prediction_set["training_year"] = training_year
        prediction_set["projection_value"] = projection_value
    except:
        continue
    strat.disconnect()

100%|█████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.72s/it]


In [14]:
strat.connect()
strat.store("recs",prediction_set)
strat.disconnect()

In [18]:
models["model"] = [pickle.dumps(x) for x in models["model"]]
models["year"] = year
models["quarter"] = quarter
models["projection_Value"] = projection_value
models["training_year"] = training_year
strat.connect()
strat.store("models",models)
strat.disconnect()